In [1]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.enums import DynamicOrderSettingsArrays
from quantfreedom.enums import *
from quantfreedom.helper_funcs import dos_cart_product
from quantfreedom.strategies.strategy import (
    strat_bt_create_ind,
    strat_evaluate,
    strat_get_current_ind_settings,
    strat_get_ind_set_str,
    strat_get_total_ind_settings,
)
from quantfreedom.simulate import sim_df_backtest
from quantfreedom.sim_order_records import sim_get_or
import requests

mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

In [2]:
candles_np = mufex_main.get_candles(symbol="BTCUSDT", timeframe="5m", candles_to_dl=3000)

It took 00 mins and 00 seconds to download 3000 candles


In [3]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
# backtest_settings = BacktestSettings(
#     divide_records_array_size_by=1,
#     gains_pct_filter=-90,
#     total_trade_filter=10,
#     upside_filter=-np.inf,
# )
backtest_settings = BacktestSettings()
dos_arrays = DynamicOrderSettingsArrays(
    entry_size_asset=np.array([0]),
    max_equity_risk_pct=np.array([9]),
    max_trades=np.array([5]),
    num_candles=np.array([100]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2, 5, 6]),
    sl_based_on_add_pct=np.array([0.01]),
    sl_based_on_lookback=np.array([20]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    sl_to_be_ze_type=np.array([0]),
    static_leverage=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5, 1]),
    trail_sl_when_pct=np.array([1, .5]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    long_or_short=LongOrShortType.Long,
    logger_bool=True,
    pg_min_max_sl_bcb=PriceGetterType.Min,
    sl_to_be_bool=False,
    z_or_e_type=ZeroOrEntryType.Nothing,
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    tp_fee_type=TakeProfitFeeType.Limit,
    trail_sl_bool=True,
)
dos_cart_arrays = dos_cart_product(
    dos_arrays=dos_arrays,
)

ExchangeSettings(
    limit_fee_pct = 0.0003,
    max_leverage = 150.0,
    market_fee_pct = 0.0009,
    mmr_pct = 0.004,
    min_leverage = 1.0,
    max_asset_size = 100.0,
    min_asset_size = 0.001,
    asset_tick_step = 3,
    position_mode = 1,
    leverage_mode = 1,
    price_tick_step = 1,
    leverage_tick_step = 2,
)


In [4]:
strat_records_df = sim_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_cart_arrays=dos_cart_arrays,
    evaluate=strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=strat_get_current_ind_settings,
    get_ind_set_str=strat_get_ind_set_str,
    get_total_ind_settings=strat_get_total_ind_settings,
    ind_creator=strat_bt_create_ind,
    logger_type=LoggerType.File,
    starting_equity=1000.0,
    static_os=static_os,
)
strat_records_df

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 6
Total order settings to test: 12
Total combinations of settings to test: 72
Total candles: 3,000
Total candles to test: 216,000


,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,to_the_upside,fees_paid,total_pnl,ending_eq
0,0,0,73.0,24,49,7.26,32.88,0.0000,1868.630,72.6343,1072.6343
1,0,1,90.0,27,63,-28.79,30.00,-0.0827,2291.562,-287.8546,712.1454
2,0,2,79.0,23,56,-37.94,29.11,-0.0000,1816.184,-379.3874,620.6126
3,0,3,92.0,25,67,-48.40,27.17,-0.3951,1995.314,-483.9846,516.0154
4,0,4,65.0,17,48,-32.65,26.15,-0.0000,2822.017,-326.5223,673.4777
...,...,...,...,...,...,...,...,...,...,...,...
67,5,7,9.0,4,5,48.38,44.44,0.0000,213.539,483.8339,1483.8339
68,5,8,9.0,5,4,64.38,55.56,0.0000,215.705,643.8183,1643.8183
69,5,9,9.0,5,4,56.67,55.56,0.0000,203.484,566.6925,1566.6925
70,5,10,9.0,4,5,60.20,44.44,0.0000,228.118,602.0338,1602.0338


In [5]:
order_records_df = sim_get_or(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_cart_arrays=dos_cart_arrays,
    evaluate=strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=strat_get_current_ind_settings,
    get_ind_set_str=strat_get_ind_set_str,
    get_total_ind_settings=strat_get_total_ind_settings,
    ind_creator=strat_bt_create_ind,
    logger_type=LoggerType.Pass,
    starting_equity=1000.0,
    static_os=static_os,
    ind_set_index=0,
    dos_index=0,
)

In [6]:
order_records_df

,ind_set_idx,or_set_idx,bar_idx,timestamp,datetime,order_status,equity,available_balance,cash_borrowed,cash_used,...,entry_size_usd,entry_price,exit_price,position_size_asset,position_size_usd,realized_pnl,sl_pct,sl_price,tp_pct,tp_price
0,0,0,100,1697657400000,2023-10-18 19:30:00,EntryFilled,1000.0000,890.4750,9381.379,109.525,...,9490.904,28264.2,NaN,0.336,9490.904,NaN,0.14,28225.7,0.8,28476.9
1,0,0,101,1697657700000,2023-10-18 19:35:00,EntryFilled,1000.0000,777.5440,18944.933,222.456,...,9786.010,28261.5,NaN,0.682,19276.914,NaN,0.13,28225.7,0.7,28473.2
2,0,0,102,1697658000000,2023-10-18 19:40:00,EntryFilled,1000.0000,660.1390,28778.782,339.861,...,10173.710,28257.3,NaN,1.042,29450.624,NaN,0.13,28225.7,0.7,28468.5
3,0,0,106,1697659200000,2023-10-18 20:00:00,StopLossFilled,909.4098,909.4098,NaN,NaN,...,NaN,NaN,28225.7,NaN,NaN,-90.5902,NaN,NaN,NaN,NaN
4,0,0,107,1697659500000,2023-10-18 20:05:00,EntryFilled,909.4098,811.4310,8392.400,97.979,...,8490.379,28234.0,NaN,0.301,8490.379,NaN,0.14,28195.0,0.8,28447.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,0,0,2888,1698493800000,2023-10-28 11:50:00,EntryFilled,1178.8933,916.4720,18497.831,262.421,...,6912.276,34188.7,NaN,0.555,19011.578,NaN,0.38,34078.1,1.2,34627.1
301,0,0,2899,1698497100000,2023-10-28 12:45:00,StopLossFilled,1072.6343,1072.6343,NaN,NaN,...,NaN,NaN,34078.1,NaN,NaN,-106.2590,NaN,NaN,NaN,NaN
302,0,0,2900,1698497400000,2023-10-28 12:50:00,EntryFilled,1072.6343,973.2560,8248.127,99.379,...,8347.506,33999.7,NaN,0.246,8347.506,NaN,0.20,33930.5,0.9,34301.3
303,0,0,2901,1698497700000,2023-10-28 12:55:00,EntryFilled,1072.6343,886.5390,15226.455,186.096,...,7164.424,34021.3,NaN,0.457,15511.930,NaN,0.22,33930.5,0.9,34328.0
